In [1]:
import os
import sys

#pyspark-shell
#os.system("pip install kafka-python")

os.environ["SPARK_HOME"]="/home/cloudera/Downloads/spark-2.4.4-bin-hadoop2.6"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HADOOP_HOME"] ="/usr/lib/hadoop/bin"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

#org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.3 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.4 pyspark-shell'




In [2]:
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json
from pyspark.sql.types import StructType, IntegerType, StringType

In [3]:
#sc = SparkContext(appName="PythonSparkStreamingKafka_RM_01")
#sc.setLogLevel("WARN")
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .appName("Spark Structured Streaming from Kafka") \
  .getOrCreate()

In [4]:
emp_data = spark \
  .readStream.format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "emps") \
  .option("startingOffsets", "latest") \
  .load() \
  .selectExpr("CAST(value AS STRING)") 

In [5]:
dept_data = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "depts") \
  .option("startingOffsets", "latest") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import *

In [7]:
dept_schema = StructType([ \
  StructField("dept_id", StringType()), StructField("department", StringType())])

In [8]:
emp_schema = StructType([ \
  StructField("id", StringType()), StructField("name", StringType()), \
  StructField("salary", FloatType()), StructField("dept_id", StringType())])

In [9]:
def parse_data_from_kafka_message(sdf, schema):
    from pyspark.sql.functions import split
    assert sdf.isStreaming == True, "DataFrame doesn't receive streaming data"
    col = split(sdf['value'], ',') #split attributes to nested array in one Column
    #now expand col to multiple top-level columns
    for idx, field in enumerate(schema): 
        sdf = sdf.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return sdf.select([field.name for field in schema])

In [10]:
emp_data = parse_data_from_kafka_message(emp_data, emp_schema)
#dept_data = parse_data_from_kafka_message(dept_data, dept_schema)

In [11]:
query = emp_data.groupBy("id").count()

In [12]:
query.writeStream \
    .queryName("emp_data") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [13]:
query.writeStream \
    .queryName("dept_data") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [14]:
spark.sql("select * from emp_data").show()

+---+-----+
| id|count|
+---+-----+
+---+-----+



In [15]:
spark.sql("select * from dept_data").show()

+---+-----+
| id|count|
+---+-----+
+---+-----+



In [ ]:
from IPython.display import display, clear_output
from time import sleep
while True:
    clear_output(wait=True)
    display(spark.sql('SELECT * FROM emp_data').show())
    sleep(1)

+-----+-----+
|   id|count|
+-----+-----+
|    x|    1|
|    f|    1|
| hivr|    1|
|    e|    2|
|spark|    1|
|    c|    2|
+-----+-----+



None